In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA
import seaborn as sb
from scipy.cluster import hierarchy
from scipy.stats import variation
from sklearn.cluster import AgglomerativeClustering, Birch
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import MiniBatchSparsePCA
from sklearn.mixture import GaussianMixture

# Load Processed Data

In [2]:
train_data = pd.read_csv('train_data_processed.csv')
test_data = pd.read_csv('test_data_processed.csv')
train_labels = np.load('train_labels.csv.npy', allow_pickle=True)
test_labels = np.load('test_labels.csv.npy', allow_pickle=True)

# Determine important genes from PCA

In [3]:
# Explains more than 80% of the variance in the data
pca = PCA(n_components=100)
scaler = StandardScaler()

train_scaled = scaler.fit_transform(train_data)
pca_train_genes = pca.fit_transform(train_scaled)

test_scaled = scaler.transform(test_data)
pca_test_genes = pca.transform(test_scaled)

all_weights = pd.DataFrame(pca.components_)
all_weights.columns = train_data.columns.values

In [4]:
important_genes = []
for i in range(50):
    important_genes.append([pd.DataFrame(np.abs(all_weights)).T.nlargest(100, i).index.values])
important_genes = np.unique(important_genes, return_counts=False)

In [5]:
important_train = train_data[important_genes]
important_test = test_data[important_genes]

In [6]:
label_mapping = {'WHO II': 2, 'WHO III': 1, 'WHO IV': 0}
mapped_labels_train = [label_mapping[x[0]] for x in train_labels]
mapped_labels_test = [label_mapping[x[0]] for x in test_labels]

# Determine Important Genes from FA

In [7]:
factors = FactorAnalysis(n_components = 100, copy = True, random_state = 15)
fa_train_genes = factors.fit_transform(train_data)
fa_test_genes = factors.transform(test_data)

In [8]:
all_weights = pd.DataFrame(factors.components_)
col_names = train_data.columns.values
all_weights.columns = col_names

In [9]:
fa_important_genes = []
temp = pd.DataFrame(np.abs(all_weights)).T
for i in range(50):
    fa_important_genes.extend(temp.nlargest(100, i).index.values)
fa_important_genes = np.unique(fa_important_genes)
fa_important_train = train_data[fa_important_genes]
fa_important_test = test_data[fa_important_genes]

# Sparse PCA

In [10]:
n = 2
sparse_pca = MiniBatchSparsePCA(n_components = n).fit(train_scaled)
sparse_train_genes = sparse_pca.transform(train_scaled)
sparse_test_genes = sparse_pca.transform(test_scaled)

# Common Functions

In [11]:
def display_pca(input_data, labels, title):
    high_scaler = StandardScaler()
    input_scaled = high_scaler.fit_transform(input_data)
    
    pca = PCA(n_components=2)
    transformed_data = pca.fit_transform(input_scaled)
    
    data_df = pd.DataFrame(data = transformed_data, columns = ['principal component 1', 'principal component 2'])
    class_labels_df = pd.DataFrame(labels)
    data_df = pd.concat([data_df, class_labels_df], axis = 1)
    data_df.columns = ['PC1', 'PC2', 'Grade']
    
    fig = plt.figure(figsize = (20, 10))
    ax = fig.add_subplot(1,2,1) 
    ax.set_title(title, fontsize = 15)
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    targets = [0, 1, 2]
    colors = ['r', 'g', 'b']
    for target, color in zip(targets,colors):
        indicesToKeep = data_df['Grade'] == target
        ax.scatter(data_df.loc[indicesToKeep, 'PC1']
                   , data_df.loc[indicesToKeep, 'PC2']
                   , c = color
                   , s = 50)
    ax.legend(targets)
    ax.grid()

In [12]:
def get_accuracy(true_labels, predicted_labels):
    accuracy = 0
    for i in range(len(true_labels)):
        if true_labels[i] == predicted_labels[i]:
            accuracy += 1
    return accuracy/len(true_labels)

In [13]:
def get_birch_model(input_data):
    birch_cluster = Birch(n_clusters=3)
    results = birch_cluster.fit_predict(input_data)
    return birch_cluster, results

In [14]:
def get_gmm_model(input_data):
    mixture_model = GaussianMixture(n_components=3, verbose=1, max_iter=1000, warm_start=True)
    predictions = mixture_model.fit_predict(input_data)
    return mixture_model, predictions

# Apply BIRCH to the three methods above

### PCA Important Genes

In [155]:
birch_cluster, results = get_birch_model(important_train)
get_accuracy(mapped_labels_train, results)

0.5958904109589042

In [156]:
results = birch_cluster.predict(important_test)
get_accuracy(results, mapped_labels_test)

0.6363636363636364

### FA Important Genes

In [157]:
birch_cluster, results = get_birch_model(fa_important_train)
get_accuracy(mapped_labels_train, results)

0.22945205479452055

In [144]:
results = birch_cluster.predict(fa_important_test)
get_accuracy(results, mapped_labels_test)

0.18181818181818182

### Sparse PCA

In [148]:
birch_cluster, results = get_birch_model(sparse_train_genes)
get_accuracy(mapped_labels_train, results)

0.476027397260274

In [147]:
results = birch_cluster.predict(sparse_test_genes)
get_accuracy(results, mapped_labels_test)

0.45454545454545453

# Other Unsupervised methods which deal with high dimensions and/overlapping

- EM (expectation maximization; apparently very good for overlapping clusters) and can assign different ways to constrain the data (diagonal, tie, etc)
- http://ceur-ws.org/Vol-1455/paper-06.pdf

# Apply EM to the three methods above

In [15]:
label_mapping = {'WHO II': 1, 'WHO III': 0, 'WHO IV': 2}
mapped_labels_train = [label_mapping[x[0]] for x in train_labels]
mapped_labels_test = [label_mapping[x[0]] for x in test_labels]

### All Data

In [76]:
# 3 gaussians used in the model
gmm, predictions = get_gmm_model(train_data)
get_accuracy(predictions, mapped_labels_train)

Initialization 0
Initialization converged: True


0.4726027397260274

In [78]:
test_predictions = gmm.predict(test_data)
get_accuracy(test_predictions, mapped_labels_test)

0.3939393939393939

In [14]:
test_predictions = gmm.predict(test_data)
get_accuracy(test_predictions, mapped_labels_test)

0.7272727272727273

### Top 100 PCs

In [70]:
pca = PCA(n_components=100)
scaler = StandardScaler()

train_scaled = scaler.fit_transform(train_data)
pca_train_genes = pca.fit_transform(train_scaled)

test_scaled = scaler.transform(test_data)
pca_test_genes = pca.transform(test_scaled)

In [71]:
gmm, predictions = get_gmm_model(pca_train_genes)
get_accuracy(predictions, mapped_labels_train)

Initialization 0
Initialization converged: True


0.5787671232876712

In [73]:
test_predictions = gmm.predict(pca_test_genes)
get_accuracy(test_predictions, mapped_labels_test)

0.30303030303030304

### PCA Important Genes

In [46]:
gmm, predictions = get_gmm_model(important_train)
get_accuracy(predictions, mapped_labels_train)

Initialization 0
Initialization converged: True


0.5513698630136986

In [47]:
test_predictions = gmm.predict(important_test)
get_accuracy(test_predictions, mapped_labels_test)

0.5454545454545454

### FA Important Genes

In [40]:
gmm, predictions = get_gmm_model(fa_important_train)
get_accuracy(predictions, mapped_labels_train)

Initialization 0
Initialization converged: True


0.5958904109589042

In [41]:
test_predictions = gmm.predict(fa_important_test)
get_accuracy(test_predictions, mapped_labels_test)

0.6060606060606061

### Sparse PCA

In [44]:
gmm, predictions = get_gmm_model(sparse_train_genes)
get_accuracy(predictions, mapped_labels_train)

Initialization 0
Initialization converged: True


0.4897260273972603

In [45]:
test_predictions = gmm.predict(sparse_test_genes)
get_accuracy(test_predictions, mapped_labels_test)

0.45454545454545453